In [1]:
import numpy
from numpy.random import randn

from openfermion.transforms._jordan_wigner import *

from openfermion.utils._trotter_exp_to_qgates import *

from openfermion.utils import (commutator, count_qubits, expectation,
                               hermitian_conjugated, normal_ordered,
                               jordan_wigner_sparse, jw_hartree_fock_state,
                               s_squared_operator, sz_operator)

from openfermion.utils._unitary_cc import (uccsd_generator,
                                           uccsd_singlet_generator,
                                           uccsd_singlet_get_packed_amplitudes,
                                           uccsd_singlet_paramsize)

In [2]:
test_orbitals = 2

single_amplitudes = randn(*(test_orbitals,) * 2)
double_amplitudes = randn(*(test_orbitals,) * 4)
print('singles')
print(single_amplitudes)
print('doubles')
print(double_amplitudes)


singles
[[-0.12559933 -1.08413724]
 [ 0.95785935  0.41714825]]
doubles
[[[[-0.56734348  0.32218425]
   [-1.3921902   1.41875146]]

  [[-0.00785519 -0.79265165]
   [ 0.98373991 -1.30957591]]]


 [[[ 0.83079796  0.79319516]
   [ 1.54594383 -1.09963432]]

  [[ 0.39225201 -0.36187252]
   [-1.71635701  0.97441173]]]]


# The OpenFermion representaion of the ansatz

In [3]:
generator = uccsd_generator(single_amplitudes, double_amplitudes)
qubit_op = jordan_wigner(generator)
qubit_op *= 1.0j

#print('openferion pre trotterized generator')
#print(generator)
print('')
print(qubit_op)


(1.0719145972781818+0j) [X0 Y1] +
(-1.0719145972781818+0j) [Y0 X1]


In [4]:
from forestopenfermion.pyquil_connector import qubitop_to_pyquilpauli
from openfermion.ops import QubitOperator

# The PyQuil representation of the operator

In [5]:
pauli_sum_representation = qubitop_to_pyquilpauli(qubit_op)
print(pauli_sum_representation)

(-1.0719145972781818+0j)*Y0*X1 + (1.0719145972781818+0j)*X0*Y1


In [6]:
from pyquil.quil import Program
from forestopenfermion import exponentiate
from pyquil.gates import *

pyquil_program = exponentiate(qubit_op)
pyquil_program = Program(X(0), pyquil_program)
print(pyquil_program)
print(len(pyquil_program))

X 0
RX(pi/2) 0
H 1
CNOT 0 1
RZ(-2.1438291945563637) 1
CNOT 0 1
RX(-pi/2) 0
H 1
H 0
RX(pi/2) 1
CNOT 0 1
RZ(2.1438291945563637) 1
CNOT 0 1
H 0
RX(-pi/2) 1

15


In [7]:
from pyquil.api import WavefunctionSimulator
from pyquil import Program

print('')
wfn = WavefunctionSimulator().wavefunction(pyquil_program)
print(wfn)


(-0.5421829373+0j)|01> + (0.840260473+0j)|10>


# The correct answer!

In [8]:
# (0.2671478502+0j)|01> + (0.9636555537+0j)|10>

In [9]:
#H0  Rx1  cX1-0  Rz1  cX1-0  H0  Rx1  Rx0  H1  cX1-0  Rz1  cX1-0  Rx0  H1

# Now begin importing trial circuit to Qforte

In [10]:
import qforte

In [11]:
#get from openferion
qforte_generator = qforte.build_from_openfermion(qubit_op)
qforte.smart_print(qforte_generator)


 Quantum operator:
(-1.0719145972781818+0j)[Y0 X1]
+ (1.0719145972781818+0j)[X0 Y1]


# Now try to exponentiate the full operator 

In [12]:
expn = qforte.trotterization.trotterize(qforte_generator)
qforte.smart_print(expn)
#expn.str()


 Quantum circuit:
[Rx0  H1  cX1-0  Rz1  cX1-0  Rx0  H1  H0  Rx1  cX1-0  Rz1  cX1-0  H0  Rx1]


In [13]:
qc2 = qforte.QuantumComputer(2)
qc2.apply_gate(qforte.make_gate('X',0,0))
#qc2.apply_gate(qforte.make_gate('X',1,1))
qc2.str()


['(0.000000 +0.000000 i) |00>',
 '(1.000000 +0.000000 i) |10>',
 '(0.000000 +0.000000 i) |01>',
 '(0.000000 +0.000000 i) |11>']

In [14]:
qc2.apply_circuit(expn)
qc2.str()

['(0.000000 +0.000000 i) |00>',
 '(-0.542183 +0.000000 i) |10>',
 '(0.840260 +0.000000 i) |01>',
 '(0.000000 +0.000000 i) |11>']

In [15]:
#(0.3510953756+0j)|0011> + (-0.6045891379+0j)|0101> + (-0.1071633537+0j)|0110> + (0.1697168339+0j)|1001> + (-0.6312331758+0j)|1010> + (-0.2691856261+0j)|1100>

In [16]:
# Now try optemizing the operators... 